In [1]:
import os
import json
import pandas as pd
import torch
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForMultipleChoice, Trainer, TrainingArguments

In [2]:
# set the file path
train_path = '/Users/hoshea/Documents/pythonProject/NLP/train.jsonl'
test_path =  '/Users/hoshea/Documents/pythonProject/NLP/eval.jsonl'

# read the file into a pandas DataFrame
# df_train = pd.read_json(train_path,lines = True)
# df_test = pd.read_json(test_path,lines = True)

In [3]:
#pip install 'transformers[torch]' accelerate -U

In [4]:
#pip install sentencepiece

In [5]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

data_files = {"train": train_path}
dataset = load_dataset('json', data_files=data_files)

# split the data
def train_test_split(dataset, test_size=0.2):
    train_test = dataset['train'].train_test_split(test_size=test_size)
    return DatasetDict({
        'train': train_test['train'],
        'test': train_test['test']
    })
dataset = train_test_split(dataset)

# import tokenizer
model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    questions = examples["Question"]
    choices = [[examples["Alternative1"][i], examples["Alternative2"][i]] for i in range(len(questions))]
    labels = [label - 1 for label in examples["Answer"]]  # 将1/2转换为0/1

    for label in labels:
        if label not in [0, 1]:
            raise ValueError(f"Unexpected label value: {label}")

    contexts = []
    for question, choice_pair in zip(questions, choices):
        for choice in choice_pair:
            contexts.append((question, choice))

    tokenized_examples = tokenizer(*zip(*contexts), truncation=True, padding='max_length', max_length=128)

    input_ids = [tokenized_examples['input_ids'][i:i+2] for i in range(0, len(tokenized_examples['input_ids']), 2)]
    attention_mask = [tokenized_examples['attention_mask'][i:i+2] for i in range(0, len(tokenized_examples['attention_mask']), 2)]

    return {
        'input_ids': torch.tensor(input_ids),
        'attention_mask': torch.tensor(attention_mask),
        'labels': torch.tensor(labels)
    }

# preprocess
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)

# load model
model = AutoModelForMultipleChoice.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/Users/hoshea/Documents/pythonProject/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/Users/hoshea/Documents/pythonProject/venv/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/11929 [00:00<?, ? examples/s]

Map:   0%|          | 0/2983 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,  # 只保留最新的检查点
    load_best_model_at_end=True,
    learning_rate=2e-5,
    per_device_train_batch_size=2,  # 由于多选任务内存消耗较大，适当减小batch size
    num_train_epochs=3,
    weight_decay=0.01,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer=tokenizer,
)

trainer.train()

# save the best model
trainer.save_model("./best_model")

/Users/hoshea/Documents/pythonProject/venv/lib/python3.12/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.689700,0.721564
2,0.488700,1.084918
3,0.169900,1.421945


In [7]:
import os
import json
import pandas as pd
import torch
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForMultipleChoice, Trainer

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

def load_eval_data(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    data = [json.loads(line) for line in lines]
    return data

eval_data = load_eval_data(test_path)

eval_df = pd.DataFrame(eval_data)

# import tokenizer and model
model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMultipleChoice.from_pretrained("./best_model")  # 使用保存好的模型路径

# Preprocess for evaluation
def preprocess_function(examples):
    questions = examples["Question"]
    choices = [[examples["Alternative1"][i], examples["Alternative2"][i]] for i in range(len(questions))]

    contexts = []
    for question, choice_pair in zip(questions, choices):
        for choice in choice_pair:
            contexts.append((question, choice))

    # Tokenize
    tokenized_examples = tokenizer(*zip(*contexts), truncation=True, padding='max_length', max_length=128)

    input_ids = [tokenized_examples['input_ids'][i:i+2] for i in range(0, len(tokenized_examples['input_ids']), 2)]
    attention_mask = [tokenized_examples['attention_mask'][i:i+2] for i in range(0, len(tokenized_examples['attention_mask']), 2)]

    return {
        'input_ids': torch.tensor(input_ids, dtype=torch.long),
        'attention_mask': torch.tensor(attention_mask, dtype=torch.long)
    }

# Build dataset for evaluation
eval_dataset = Dataset.from_pandas(eval_df)
eval_dataset = eval_dataset.map(preprocess_function, batched=True, remove_columns=eval_df.columns.tolist())

trainer = Trainer(model=model, tokenizer=tokenizer)

predictions = trainer.predict(eval_dataset)

predicted_logits = np.array(predictions.predictions)
predicted_labels = np.argmax(predicted_logits, axis=1).tolist()


output = pd.DataFrame({
    'ID': eval_df['Id'],
    'Target': [label + 1 for label in predicted_labels]  # 将标签从0/1转换回1/2
})

# save the result
output.to_csv('eval_predictions.csv', index=False)

print("Predictions saved to eval_predictions.csv")


/Users/hoshea/Documents/pythonProject/venv/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/4261 [00:00<?, ? examples/s]

Predictions saved to eval_predictions.csv
